In [13]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib_fontja

## データの読み込み・データの前処理

In [25]:
### パラメータを設定（Excelデータを見ながら）
data_path = '../data/private/demo_crosstab.xlsx'
sheet_name = 'N数'
col_start = 1
col_end = 21
row_start = 1
row_end = 21
drop_col_list = []
drop_row_list = []
drop_all_zero_col = False
drop_all_zero_rost = False

### データの読み込み
df_full = pd.read_excel(data_path, sheet_name=sheet_name, header=row_start-1, index_col=col_start-1)

### データ全体からテーブル部分を抽出
df_subset = df_full.iloc[:row_end-row_start, :col_end-col_start]

### 特定の列を削除
if len(drop_col_list) > 0:
    df_subset.drop(columns=df_subset.columns[np.array(drop_col_list)-col_start-1], inplace=True)

### 特定の行を削除
if len(drop_row_list) > 0:
    df_subset.drop(index=df_subset.index[np.array(drop_row_list)-row_start-1], inplace=True)

### 全ての値がゼロの列を削除
if drop_all_zero_col:
    df_subset = df_subset.loc[:, (df_subset != 0).any(axis=0)]

### 全ての値がゼロの行を削除
if drop_all_zero_rost:
    df_subset = df_subset.loc[(df_subset != 0).any(axis=1), :]

display(df_subset.head())

,model020,model021,model022,model023,model024,model025,model026,model027,model028,model029,model030,model031,model032,model033,model034,model035,model036,model037,model038,model039
model020,0,30,0,14,0,0,0,0,0,0,0,0,26,30,0,0,0,20,0,8
model021,5,0,0,0,0,11,14,0,0,28,18,5,25,0,26,23,0,9,0,0
model022,0,0,0,12,11,0,28,0,22,0,0,0,0,0,0,23,9,30,0,7
model023,16,0,0,0,18,0,24,10,11,0,0,25,7,14,0,30,23,0,23,0
model024,0,6,16,0,0,25,5,0,0,20,25,5,0,30,26,0,19,0,12,0


In [24]:
### シリーズモデルを１つにまとめる
series_models = ['model020', 'model022', 'model025'] 
series_name = 'seriesA'
df_subset[series_name] = df_subset[series_models].sum(axis=1)
df_subset.loc[series_name] = df_subset.loc[series_models, :].sum(axis=0)

display(df_subset)

,model020,model021,model022,model023,model024,model025,model026,model027,model028,model029,...,model031,model032,model033,model034,model035,model036,model037,model038,model039,seriesA
model020,0,30,0,14,0,0,0,0,0,0,...,0,26,30,0,0,0,20,0,8,0
model021,5,0,0,0,0,11,14,0,0,28,...,5,25,0,26,23,0,9,0,0,16
model022,0,0,0,12,11,0,28,0,22,0,...,0,0,0,0,23,9,30,0,7,0
model023,16,0,0,0,18,0,24,10,11,0,...,25,7,14,0,30,23,0,23,0,16
model024,0,6,16,0,0,25,5,0,0,20,...,5,0,30,26,0,19,0,12,0,41
model025,30,29,18,7,27,0,19,16,0,7,...,0,0,0,24,9,0,23,16,21,48
model026,0,0,0,5,0,0,0,0,13,0,...,0,27,0,0,0,29,0,6,0,0
model027,0,22,0,0,0,17,28,0,12,23,...,0,30,0,0,0,0,28,27,0,17
model028,16,0,6,0,0,0,0,25,0,0,...,23,0,28,30,30,0,5,22,0,22
model029,0,0,14,0,7,0,23,27,24,0,...,9,5,0,19,27,18,0,18,0,14


## データ分析

In [29]:
df = df_subset.copy()

### 競合車を分析するターゲットモデルを指定
target_model = "model025"

### 分析1.1 直接競合の特定|ターゲットモデルを現保有の人が比較した車

In [51]:
### 統計量を計算
total_all = df.values.sum() ### 全セルの合計（全体人数）
total_row = df.sum(axis=1)  ### 各行の合計（比較された人数）
total_col = df.sum(axis=0)  ### 各列の合計（購入された人数）
expected = np.outer(total_row, total_col) / total_all  ### 期待値 = 外積 / 全体合計
expected = pd.DataFrame(expected, index=df.index, columns=df.columns) ### DataFrameに変換

### ターゲットモデルの現保有者の比較検討車の部分を抽出
df_target = df[target_model]
df_target = df_target[df_target > 0].copy()

In [53]:
df_target

model021    11
model024    25
model027    17
model030    10
model033     6
model035    26
model036    23
model037    29
model039    14
Name: model025, dtype: int64

In [52]:
for model_name, count in df_target.items():
    # AとBの2×2クロス表を構築（検定のため）
    r = df.loc[model_name]
    c = df[target_model] 
    sub = pd.DataFrame([
        [count, c.sum() - count],
        [r.sum() - count, total_all - r.sum() - c.sum() + count]
    ])

In [54]:
sub

,0,1
0,14,147
1,206,3160
